# Visualizing change in reservoir surface water extent using the OPERA DSWx product with a time slider
---

**This notebook serves as a visualization tool using DSWx B01_WTR to illustrate change of Lake Mead, NV during 2022.**

In [ ]:
# notebook dependencies
import os
import math
import xarray as xr
import rasterio as rio
import rioxarray as rioxr
import hvplot.xarray
import geoviews as gv
import pyproj
from pyproj import Proj
import numpy as np
import pandas as pd
import geopandas as gpd
import holoviews as hv
import panel.widgets as pnw
import datetime
from datetime import datetime

from bokeh.models import FixedTicker
hv.extension('bokeh')

import warnings
warnings.filterwarnings('ignore')

os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

---
## **Data Information Input**

In the code cell below, the user should specify the:
* Dates of interest <br>
* Data directory<br>
* Band of interest<br>
* Path to shapefile to create mask<br><br>

**<font color='red'>Note: The cell below is the only code in the notebook that should be modified. </font>**

In [ ]:
# Access all 2022 provisional products from S3 bucket
data_dir = [match for match in sorted(open("aux_files/T11SQA_manifest.txt","r").read().split("\n")) if '2022' in match]
for i,path in enumerate(data_dir):
   if path != '': # discard empty entries
       if path[91] == 'L': # Landsat 8 filenames
           data_dir[i] = path+path[32:101]+'_'
       if path[91] == 'S': #Sentinel-2 filenames
           data_dir[i] = path+path[32:102]+'_'
# Landsat filenames are 1 character shorter than Sentinel-2 filenames

# Extract date information from filename
dates = [path[57:65] for path in data_dir]

# Change this to the desired band for visualization
band = 'B01_WTR'

# Path to shapefile used to create mask of pixels close to Lake Mead
shapepath = 'aux_files/bufferlakebnds/bufferlakebnds.shp'


***

### **<font color='red'> -- Do not modify any of the code below -- </font>**

In [ ]:
# Functions

def file_dataframe(data_dir:str,datelist:str):
    '''
    Returns dataframe with data_dir to DSWx layers for a given list of dates.
        Parameters:
                data_dir (str): Working directory for each date
                datelist (str): List of dates
        Returns:
                dswx_file_df (Pandas DataFrame): List of data_dir to layer for each date
    '''
    fileDF = []
    for i,date in enumerate(dates):
        paths = f"{data_dir[i]}%s.tif"
        B01_WTR = paths%'B01_WTR'
        B02_BWTR = paths%'B02_BWTR'
        B03_CONF = paths%'B03_CONF'
        B04_DIAG = paths%'B04_DIAG.tif'
        B05_WTR_1 = paths%'B05_WTR-1.tif'
        B06_WTR_2 = paths%'B06_WTR-2.tif'
        B07_LAND = paths%'B07_LAND.tif'
        B08_SHAD = paths%'*B08_SHAD.tif'
        B09_CLOUD = paths%'*B09_CLOUD.tif'
        B10_DEM = paths%'DEM.tif'
        fileDF.append([date,B01_WTR,B02_BWTR,B03_CONF,B04_DIAG,B05_WTR_1,B06_WTR_2,B07_LAND,B08_SHAD,B09_CLOUD,B10_DEM])
    fileDF = pd.DataFrame(fileDF,columns = ['Date', 'B01_WTR', 'B02_BWTR', 'B03_CONF', 'B04_DIAG', 'B05_WTR_1', 'B06_WTR_2', \
        'B07_LAND', 'B08_SHAD', 'B09_CLOUD', 'B10_DEM']).astype('string')
    return fileDF

def stack_layers(files:str,datelist:str):
    '''
    Returns geocube with one band over time stacked into one multi-dimensional array.
            Parameters:
                    files (str): data_dir to band for each date
                    datelist (list): List of dates
            Returns:
                    layerStack (xarray Dataset): Geocube with band stacked in time
                    crs (int): Coordinate Reference System corresponding to band
    '''
    layerStack = []; layerS = []; layerStack_ = [];
    for i,d in enumerate(datelist):
        time = datetime.strptime(d,'%Y%m%d')
        if i == 0:
            layerStack_ = xr.open_rasterio(files[i])
            crs = pyproj.CRS.to_epsg(pyproj.CRS.from_proj4(layerStack_.crs))
            layerStack = layerStack_.squeeze(drop=True)
            layerStack = layerStack.to_dataset(name='z')
            layerStack.coords['time'] = np.array(time)
            layerStack = layerStack.rename({'x':'longitude', 'y':'latitude'})
            layerStack = layerStack.expand_dims(dim='time')
        else:
            layerS = xr.open_rasterio(files[i])
            layerS = layerS.squeeze(drop=True)
            layerS = layerS.to_dataset(name='z')
            layerS.coords['time'] = np.array(time)
            layerS = layerS.rename({'x':'longitude', 'y':'latitude'})
            layerS = layerS.expand_dims(dim='time')
            layerStack = xr.concat([layerStack,layerS], dim='time')
    return layerStack, crs

def buffer_mask(shapefile:str):
    '''
    Returns masked data based on buffered shapefile.
            Parameters:
                    shapefile (str): Path to buffered shapefile
            Returns:
                    masked (xarray Dataset): Geocube of masked data
    '''
    shp = gpd.read_file(shapefile)
    masked = data.rio.clip(shp.geometry)
    masked = masked.where(masked['z'] != 255.)
    return masked

### **<font color='red'> -- Do not modify any of the code above -- </font>**

---
## **1. Prepare the Geocube: Create the file dataframe, multidimensional dataset, and mask**

In [ ]:
# Create dataframe of paths to bands for each date
dswx_file_df = file_dataframe(data_dir,dates)
# Inspect the dataframe
dswx_file_df

In [ ]:
# Stack dates to create a multidimensional "geocube" for chosen band
data, crs = stack_layers(dswx_file_df[band], dates)
# Inspect the geocube dataset created
data

Only include pixels near Lake Mead by creating mask from buffered shapefile based on a [2003 USGS shapefile](https://pubs.usgs.gov/of/2009/1150/gis/basemap/lakebndsmeta.htm). This shapefile is created using the optional [prep_shapefile](link) notebook provided in this repository.

In [ ]:
# Create a masked geocube
masked_data = buffer_mask(shapepath)
# Inspect the masked dataset
masked_data

---
## **2. Visualize surface water extent of Lake Mead during 2022**

In [ ]:
# Create a basemap
base = gv.tile_sources.EsriImagery.opts(width=1000, height=1000, padding=0.1)

dswxPalette = ["#2000ff","#04fc04","#ffffff","#ffffff","#ffffff","#ffffff","#ffffff","#ffffff","#7f7f7f"]

masked_data_z = masked_data.z.where(masked_data.z>0)

masked_data_slider = masked_data_z.interactive.sel(time=pnw.DiscreteSlider).hvplot(x='longitude', 
                                                                                       y='latitude', 
                                                                                       crs=crs, 
                                                                                       kind='image', 
                                                                                       rasterize=True, 
                                                                                       dynamic=True,
                                                                                       cmap=dswxPalette, 
                                                                                       aspect='equal', 
                                                                                       frame_width=600, 
                                                                                       frame_height=600).opts(active_tools=['wheel_zoom'],
                                                                                                              xlabel='Longitude', 
                                                                                                              ylabel='Latitude',  
                                                                                                              clim=(1,9), 
                                                                                                              colorbar_opts={'ticker': FixedTicker(ticks=[0, 1, 2, 9])}, 
                                                                                                              alpha=0.9)
masked_data_slider * base